# Ficha de Análise Sintática (Yacc)

## Introdução

A análise sintática, ou "parsing" em inglês, refere-se ao processo de analisar um pedaço de texto ou código para determinar a sua estrutura e significado. Em computação, o parsing é usado para converter código escrito numa linguagem de programação de alto nível para código executável pela máquina, por exemplo. Na linguística, o parsing é usado para analisar frases em linguagem natural e determinar a sua estrutura sintática.

Em Python, podemos fazer análise sintática de várias formas. A que iremos utilizar nas aulas, tal como na análise léxica, recorre ao módulo Ply, mais especificamente à ferramenta Yacc.

O Yacc usa uma técnica de *parsing* conhecida como LR-parsing (ou *shift-reduce* parsing). O L significa que o *input* é lido da esquerda para a direita e o R significa que as regras da gramática são "reduzidas" da direita para a esquerda.

Dentro do módulo Ply, usamos a ferramenta `yacc.py` para fazer análise sintática.

In [ ]:
import ply.yacc as yacc

Para fazer análise sintática, precisamos primeiro de fazer análise léxica. Assim, vamos pegar no exemplo usado na ficha anterior e extendê-lo para realizar também o *parsing* de expressões aritméticas, passando a funcionar como uma calculadora.

In [ ]:
import ply.lex as lex

tokens = (
    'NUMBER',
    'PLUS',
    'MINUS',
    'TIMES',
    'DIVIDE',
    'LPAREN',
    'RPAREN'
)

t_PLUS = r'\+'
t_MINUS = r'\-'
t_TIMES = r'\*'
t_DIVIDE = r'\/'
t_LPAREN = r'\('
t_RPAREN = r'\)'

def t_NUMBER(t):
    r'\d+'
    t.value = int(t.value)
    return t

t_ignore = ' \n\t'

def t_error(t):
    print(f"Carácter ilegal {t.value[0]}")
    t.lexer.skip(1)

lexer = lex.lex()

O parser precisa de ter acesso aos tokens. Caso o tokenizer esteja definido noutro ficheiro, devemos importar a variável.

```py
from [nome_do_tokenizer] import tokens
```

Depois, apenas precisamos de definir as regras da gramática:

In [ ]:
def p_expression_plus(p):
    'expression : expression PLUS term'
    p[0] = p[1] + p[3]

def p_expression_minus(p):
    'expression : expression MINUS term'
    p[0] = {"currentexp":(p[1],p[3])}
    print(p[1] - p[3])

def p_expression_term(p):
    'expression : term'
    p[0] = p[1]

def p_term_times(p):
    'term : term TIMES factor'
    p[0] = p[1] * p[3]

def p_term_div(p):
    'term : term DIVIDE factor'
    p[0] = p[1] / p[3]

def p_term_factor(p):
    'term : factor'
    p[0] = p[1]

def p_factor_num(p):
    'factor : NUMBER'
    p[0] = p[1]

def p_factor_expr(p):
    'factor : LPAREN expression RPAREN'
    p[0] = p[2]

def p_error(p):
    print("Erro sintático no input!")

# expression : term
#           | expression PLUS term
#           | expression MINUS term
# term : term TIMES factor
#     |term DIVIDE factor
#     | factor
# factor : NUMBER
#        | LPAREN expression RPAREN

Cada função corresponde ao nome de uma regra da
gramática, representada por um símbolo não-terminal do lado esquerdo e uma sequência de não-terminais e *tokens* do lado direito.

Caso um não-terminal seja definido à custa de outros não-terminais, os seus valores podem ser obtidos através de `p[i]`, em que `i` é a posição do não-terminal na regra. O valor dos terminais é o valor definido no *tokenizer*.

Depois de executar a função, o valor do não-terminal à esquerda irá corresponder ao valor de `p[0]`.

Por exemplo, em

```py
def p_expression_plus(p):
    'expression : expression PLUS term'
    #   ^            ^        ^    ^
    #  p[0]         p[1]     p[2] p[3]

    p[0] = p[1] + p[3]
```

o valor de `p[2]` será "+", e o valor de `p[3]` será o valor de `p[0]` atribuído na regra `p_term_` correspondente.

A primeira regra definida determina o símbolo inicial da gramática. Neste caso é `expression`, o que significa que o *input* dado ao parser será reduzido até esta regra ser atingida ou até haver um erro do qual o parser não consiga recuperar.

Para facilitar a leitura, as regras podem também ser definidas do seguinte modo:

In [ ]:
def p_expression(p):
    '''expression : expression PLUS term
                  | expression MINUS term'''
    if p[2] == '+':
        p[0] = p[1] + p[3]
    elif p[2] == '-':
        p[0] = p[1] - p[3]

Depois de termos estas regras definidas, podemos correr o nosso *parser*.

In [ ]:
parser = yacc.yacc()

while s := input('calc > '):
   result = parser.parse(s)
   print(result)

No caso de termos uma regra como:

```py
expression : expression PLUS expression
           | expression MINUS expression
           | expression TIMES expression
           | expression DIVIDE expression
           | LPAREN expression RPAREN
           | NUMBER
```

se dermos ao *parser* uma expressão como "5 + 2 * 5", quando ele já tiver lido "5 + 2", não saberá se primeiro deve reduzir a adição ou ler o *token* da multiplicação, criando um conflito *shift/reduce*.

Em vez de estarmos a definir duas regras diferentes, como fizemos acima, podemos definir a precedência de cada token/terminal da seguinte forma:

In [ ]:
precedence = (
    ('left', 'PLUS', 'MINUS'),
    ('left', 'TIMES', 'DIVIDE'),
)

Agora, como o *token* da multiplicação tem maior precedência do que o da adição, vai haver um *shift* em vez de uma redução no exemplo acima, sem ambiguidades.

Para além de precedência à esquerda, podemos definir precedência à direita (útil para operadores unários, como a inversão de sinal) ou precedência não associativa, na qual estamos a dizer que não podemos combinar operadores daquele tipo.

In [ ]:
precedence = (
    ('nonassoc', 'LESSTHAN', 'GREATERTHAN'),  # Nonassociative operators (a < b < c)
    ('left', 'PLUS', 'MINUS'),
    ('left', 'TIMES', 'DIVIDE'),
    ('right', 'UMINUS'),            # Unary minus operator (-5)
)

A documentação completa do Yacc pode ser consultada em: https://ply.readthedocs.io/en/latest/ply.html#yacc

## Exercícios em papel

### 1) Especificação de gramáticas - exercício básico
Considere as seguintes frases:
```
eu= Mara Maria Mendes Almeida.
Eu  =Antonio Cunha Rego.
ELE=Antonio Silva.
```

Formalize estas frases escrevendo uma GIC (Gramatica independente de contexto).
```
tokens = (Token1,...)
GIC = p1:...
```


### 2) Especificação de gramática para DSL bancaria

Escreva uma GIC que formalize a linguagem seguinte:
```
DEPOSITAR 1000 NA conta1;
LEVANTAR 200 DA conta1;
TRANSFERIR 300 DA conta1 PARA conta2;
CONSULTAR SALDO DA conta1;
```
São suportadas 3 operações diferentes:
*   DEPOSITAR
*   LEVANTAR
*   TRANSFERIR
*   CONSULTAR




## Exercícios yacc

### Listas


(a) Com base no seguinte lexer:
```
# listas_lex.py
# Listas heterogéneos: inteiros e alfanuméricos
import ply.lex as lex

tokens = ['PA', 'PF', 'VIRG', 'NUM', 'ALFANUM']

t_PA = r'\['
t_PF = r'\]'
t_VIRG = r','
t_NUM = r'\d+'
t_ALFANUM = r'[A-Za-z]\w*'

t_ignore = " \t\n"

def t_error(t):
    print('Caráter ilegal: ', t.value[0])
    t.lexer.skip(1)

lexer = lex.lex()
```
Construa um analisador sintatico utilizando o yacc, que reconheça as seguintes frases de exemplo.

```
[78,teste,2]

[]

[s,s]

[2,1]
```

(b) Recolha a soma dos elementos do tipo inteiro e imprima-os após processar cada frase.

In [ ]:
from listas_lex import tokens
import ply.yacc as yacc

def p_expression_lista_vazia(p):
    'lista : PA PF'
    print(f"Lista Vazia: {p[1], p[2]}")

def p_expression_lista(p):
    'lista : PA elementos PF'
    print(f"Lista: {p[1], p[2], p[3]}")


def p_expression_elementos_unico(p):
    'elementos : elemento'
    print(f"Elementos unico: {p[1]}")
    p[0] = p[1]

def p_expression_elementos_varios(p):
    'elementos : elementos VIRG elemento'
    print(f"Elementos varios: {p[1], p[2], p[3]}")

def p_expression_elemento_num(p):
    'elemento : NUM'
    print(f"Elemento numero: {p[1]}")
    p[0] = p[1]

def p_expression_elemento_alfanum(p):
    'elemento : ALFANUM'
    print(f"Elemento alfanumerico: {p[1]}")
    p[0] = p[1]

def p_error(p):
    print(f'Erro na expressão: {p.value}')

parser = yacc.yacc()

parser.parse('[78,teste,2]')
# def p_expression_plus(p):
#     'expression : expression PLUS term'
#     p[0] = p[1] + p[3]

### 1. Lista de compras

Constroi um analisador sintático capaz de ler uma lista de compras e gerar uma estrutura de dados (como um dicionário, por exemplo) com a informação sobre cada produto que consta da lista, para além do custo total das compras.

Exemplo de uma lista de compras, com categorias. Cada produto tem um índice na lista, nome, preço individual e quantidade:

---
```
CARNE :
  - 1 :: Bife :: 10.00 :: 1;
  - 2 :: Panado :: 5.00 :: 4;
  - 3 :: Hambúrguer :: 8.00 :: 3;
  - 4 :: Almôndegas :: 7.00 :: 5;

BEBIDA :
  - 5 :: Água :: 0.40 :: 16;
  - 6 :: Sumo :: 1.50 :: 9;
  - 7 :: Refrigerante :: 1.80 :: 10;

FRUTA :
  - 8 :: Maçã :: 0.60 :: 20;
  - 9 :: Banana :: 0.50 :: 15;
  - 10 :: Laranja :: 0.80 :: 18;
  - 11 :: Pêssego :: 0.70 :: 22;
  - 12 :: Uva :: 0.90 :: 17;

LEGUMES :
  - 13 :: Alface :: 1.00 :: 25;
  - 14 :: Tomate :: 0.75 :: 23;
  - 15 :: Cebola :: 0.50 :: 28;
  - 16 :: Batata :: 0.30 :: 30;
  - 17 :: Cenoura :: 0.40 :: 26;

PASTELARIA :
  - 18 :: Bolo de Chocolate :: 3.50 :: 1;
  - 19 :: Croissant :: 1.20 :: 14;
  - 20 :: Pastel de Nata :: 1.00 :: 5;
  - 21 :: Donut :: 0.80 :: 13;
```

EXTRA :

- validar que não existem produtos duplicados
- validar que não existem categorias duplicadas

---

### 2. Linguagem do filesystem

Construa um analisador sintatico que recebe uma frase válida da liguagem dos filesystem e produz como output comandos do sistema que permitem construir o filesystem no computador do utilizador. A linguagem suporta a representação de estrutura de pastas e ficheiros.

Exemplo de input :

```
PL { doc {} , aulas { 03-33 { f1,f2,f3} } }

```

Exemplo de output :
```
mkdir PL
cd PL
mkdir doc
mkdir aulas
cd aulas
mkdir 03-33
cd 03-33
touch f1
touch f2
touch f3
cd ..
cd ..
cd ..
```

Sendo que a ideia será ter um programa funcional e utilizável, construa o analisador de maneira a receber um ficheiro como argumento para ser processado.

### 3. Tabela para CSV

(a) Define um parser que converte uma tabela, escrita em formato textual, para um ficheiro CSV.
Considere que a primeira linha da tabela apenas permite palavras.
Exemplo de uma tabela em formato textual:

---
```
| Nome        | Idade | Género      | Altura | Ocupação         |
|-------------|-------|-------------|--------|------------------|
| Alex        | 32    | SemInfo | 1.72   | Desenvolvimento  |
| Avery       | 29    | Feminino    | 1.85   | Marketing        |
| Casey       | 27    | Masculino   | 1.80   | Apoio ao cliente |
| David       | 40    | Masculino   | 1.76   | Gerente          |
| Emily       | 31    | Feminino    | 1.62   | Designer         |
| Frank       | 25    | Masculino   | 1.81   | Apoio ao cliente |
| Grace       | 38    | Feminino    | 1.68   | Vendas           |
| Harper      | 33    | SemInfo | 1.70   | Desenvolvimento  |
| Ivan        | 30    | Masculino   | 1.77   | Apoio ao cliente |
| Jane        | 29    | Feminino    | 1.74   | Designer         |
| Kim         | 35    | Feminino    | 1.79   | Gerente          |
| Liam        | 28    | Masculino   | 1.83   | Marketing        |
| Morgan      | 26    | SemInfo | 1.67   | Desenvolvimento  |
| Nathan      | 34    | Masculino   | 1.76   | Apoio ao cliente |
| Olivia      | 36    | Feminino    | 1.75   | Gerente          |
| Pat         | 37    | SemInfo | 1.78   | Designer         |
| Quinn       | 27    | SemInfo | 1.68   | Marketing        |
| Rachel      | 39    | Feminino    | 1.69   | Desenvolvimento  |
| Sam         | 23    | SemInfo | 1.70   | Apoio ao cliente |
| Taylor      | 31    | SemInfo | 1.81   | Vendas           |
```
---

(b) Permita que o parser consiga também converter uma tabela para um ficheiro JSON.

```
[
  {Nome:"nome",Idade:12,Genero:"asd",Altura:1,Ocupacao:"alguma"},
  ....
]
```

### 4. Árvores binárias

Considera que uma árvore binária pode ser representada segundo um formato textual em que `()` representa uma árvore vazia e `( e l r )` representa uma árvore com raiz `e`, uma árvore `l` à sua esquerda e uma árvore `r` à sua direita. Por exemplo, `( 5 (3 (1 () ()) ()) (8 () (10 () ())))` representa uma árvore de raiz 5, com uma árvore de raiz 3 à sua esquerda e uma árvore de raiz 8 à sua direita.

Define um analisador sintático capaz de converter esta representação textual numa árvore binária definida em Python. Para isso, deves definir uma classe em Python que represente uma árvore binária.

### 5. Markdown para HTML

A partir do analisador léxico definido na ficha lex para um ficheiro Markdown, define agora um parser que converte um ficheiro Markdown num ficheiro HTML.

1. Suporte títulos e subtítulos (opcionalmente suporte qualquer número de títulos e subtítulos mapeando para h3,h4,h5,...);
2. Suporte bold e itálico;
3. Suporte code snippets;
4. Suporte imagens;
5. Suporte para listas apenas com um nível (opcionalmente expanda para listas com aninhamento).

- **Nota:** [Conversor online para ajuda](https://markdowntohtml.com/)

### 6. Criação de AST para linguagem de programação incompleta/parcial

Desenvolva um analisador léxico e sintático utilizando o **PLY** (ply.lex e ply.yacc) para definir uma gramática parcial de uma linguagem de programação simples.

Implemente um programa em Python que:

- Utilize **PLY** para realizar a análise léxica e sintática.
- Defina um conjunto limitado de tokens e regras gramaticais que permitam a interpretação de:
  - Atribuições de variáveis.
  - Expressões aritméticas (adição, subtração, multiplicação, divisão, parêntesis e números inteiros).
  - Instruções de impressão (`print`).

- Construa uma **Árvore de Sintaxe Abstrata (AST)** representando a estrutura do programa.

## Requisitos da Linguagem

A gramática a ser implementada deverá suportar os seguintes elementos:

1. **Atribuição de Variáveis**  
   - **Sintaxe:**  
     ```
     id = expressão;
     ```
   - **Exemplo:**  
     ```
     x = 3 + 4 * 2;
     ```

2. **Expressões Aritméticas**  
   - Deve suportar números inteiros, identificadores e operações aritméticas básicas: `+`, `-`, `*`, `/`.
   - Respeitar a precedência dos operadores.
   - Permitir o uso de parêntesis para agrupar expressões.

3. **Instrução de Impressão**  
   - **Sintaxe:**  
     ```
     print(expressão);
     ```
   - **Exemplo:**  
     ```
     print(x + 2);
     ```

4. **Construção da AST**  
   - Para cada instrução e expressão, construir um nó na AST que represente a operação.
   - **Exemplo de representação de uma atribuição:**

     ```
     x = 3 + 4 * 2;
     ```
      Produz a seguinte AST:
     ```
     ('assign', 'x', ('binop', '+', ('num', 3), ('binop', '*', ('num', 4), ('num', 2))))
     ```
5. **Extra: Definir função de travessia para a AST**
